## Automated ML

Importing all needed dependencies to complete the project.

In [48]:
import requests
import json
import logging
import joblib
from pprint import pprint
import pandas as pd
from sklearn.model_selection import train_test_split

import azureml.core
from azureml.core.workspace import Workspace
from azureml.core.experiment import Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.core.compute_target import ComputeTargetException
from azureml.core.dataset import Dataset
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.widgets import RunDetails
from azureml.core.model import Model, InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.automl.core.shared import constants
from azureml.core.environment import Environment

SDK version: 1.47.0


## Workspace and Experiment

In [49]:
ws = Workspace.from_config()
# choose a name for experiment
experiment_name = "automl_maternal_health_experiment"
experiment = Experiment(ws,experiment_name)

Workspace name:  quick-starts-ws-219220
Subscription id:  81cefad3-d2c9-4f77-a466-99a7f541c7bb
Resource group: 
aml-quickstarts-219220


## AmlCompute cluster

In [51]:
# Choose a name for the cluster
cpu_cluster_name = 'cluster-yum'

try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    print('Creating a new compute cluster...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2', min_nodes=1, max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Creating a new compute cluster...
InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded..................
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 1, 'targetNodeCount': 1, 'nodeStateCounts': {'preparingNodeCount': 1, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2022-12-19T11:04:38.694000+00:00', 'errors': None, 'creationTime': '2022-12-19T11:03:21.964033+00:00', 'modifiedTime': '2022-12-19T11:03:28.730239+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 1, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT1800S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [52]:
# validate dataset is in store
key = "maternal-health"
description_text = "Maternal health"


if key in ws.datasets.keys():
    dataset = ws.datasets[key]
    print('The Dataset was found')
else:
    # Create AML Dataset and register it into Workspace
    data_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00639/Maternal Health Risk Data Set.csv"
    dataset = Dataset.Tabular.from_delimited_files(data_url)
    #Register Dataset in Workspace
    dataset = dataset.register(workspace = ws,name = key,description = description_text)

df = dataset.to_pandas_dataframe()
print(df.head())

## AutoML Configuration

In [55]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes" : 30,
    "enable_early_stopping" : True,
    "primary_metric":'accuracy',
    "n_cross_validations":4,
    "max_concurrent_iterations":4,
    "verbosity": logging.INFO
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target = compute_target,
                            task = 'classification',
                            training_data=dataset,                         
                            label_column_name="RiskLevel",
                            **automl_settings
                            )

In [56]:
# Submit your experiment
run = experiment.submit(automl_config, show_output=True)
#run.wait_for_completion()

Submitting remote run.
No run_configuration provided, running on cluster-cpu with default configuration
Running on remote compute: cluster-cpu


Experiment,Id,Type,Status,Details Page,Docs Page
automl_heart_failure_experiment,AutoML_a8026d04-0a63-4621-bb03-ffdaa17023ae,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

********************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

**********************************************************************************

In [57]:
print("Run Status: ",run.get_status())

Run Status:  Completed


## Run Details

In the cell below, use the `RunDetails` widget to show the different experiments.

In [58]:
RunDetails(run).show()
run.wait_for_completion (show_output = True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
automl_heart_failure_experiment,AutoML_a8026d04-0a63-4621-bb03-ffdaa17023ae,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

********************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

********************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more about high cardinality feat

{'runId': 'AutoML_a8026d04-0a63-4621-bb03-ffdaa17023ae',
 'target': 'cluster-cpu',
 'status': 'Completed',
 'startTimeUtc': '2022-12-19T11:07:36.377922Z',
 'endTimeUtc': '2022-12-19T11:20:50.062782Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '4',
  'target': 'cluster-cpu',
  'AMLSettingsJsonString': '{"path":null,"name":"automl_heart_failure_experiment","subscription_id":"81cefad3-d2c9-4f77-a466-99a7f541c7bb","resource_group":"aml-quickstarts-219220","workspace_name":"quick-starts-ws-219220","region":"southcentralus","compute_target"

## Best Model

In the cell below, get the best model from the automl experiments and display all the properties of the model.

In [59]:
best_run,fitted_model = run.get_output()

In [61]:
# print(best_run.get_metrics())
# print(best_run.get_details())

{'matthews_correlation': 0.6955860130073342, 'recall_score_macro': 0.8298182847441322, 'norm_macro_recall': 0.6596365694882644, 'weighted_accuracy': 0.8912802402919023, 'AUC_micro': 0.9130237724210697, 'AUC_weighted': 0.9042427741227176, 'recall_score_weighted': 0.8661711711711712, 'f1_score_micro': 0.8661711711711712, 'accuracy': 0.8661711711711712, 'average_precision_score_macro': 0.8864099646482893, 'recall_score_micro': 0.8661711711711712, 'precision_score_macro': 0.8677434818213142, 'average_precision_score_weighted': 0.9111387511467616, 'f1_score_macro': 0.83813450979573, 'balanced_accuracy': 0.8298182847441322, 'precision_score_micro': 0.8661711711711712, 'precision_score_weighted': 0.8733931603325003, 'average_precision_score_micro': 0.9124297611447804, 'AUC_macro': 0.9042427741227177, 'f1_score_weighted': 0.8604284204389155, 'log_loss': 0.397482912569034, 'accuracy_table': 'aml://artifactId/ExperimentRun/dcid.AutoML_a8026d04-0a63-4621-bb03-ffdaa17023ae_48/accuracy_table', 'con

In [65]:
# Save the best model
best_run.register_model(model_name = 'automl-best-model-yum.pkl',model_path = './outputs/')

Model(workspace=Workspace.create(name='quick-starts-ws-219220', subscription_id='81cefad3-d2c9-4f77-a466-99a7f541c7bb', resource_group='aml-quickstarts-219220'), name=automl-best-model.pkl, id=automl-best-model.pkl:4, version=4, tags={}, properties={})

In [66]:
# best_run.get_file_names()
# # Download the yaml file that includes the environment dependencies
# best_run.download_file('outputs/conda_env_v_1_0_0.yml', 'env.yml')
# # Download the model file
# best_run.download_file('outputs/model.pkl', 'Automl_model.pkl')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

In the cell below, register the model, create an inference config and deploy the model as a web service.

In [68]:
# Registring the best model
model = run.register_model(model_name='automl-best-model-yum.pkl')
print(run.model_id)

# Get automl environment with its dependencies
environment = Environment.get(ws, "AzureML-AutoML")

environment = best_run.get_environment()
entry_script='inference/scoring.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', entry_script)

inference_config = InferenceConfig(entry_script = entry_script, environment = environment)

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                                    memory_gb = 1, 
                                                    auth_enabled= True, 
                                                    enable_app_insights= True)

service = Model.deploy(ws, "aciservice", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)

automl-best-model.pkl
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-12-19 12:23:55+00:00 Creating Container Registry if not exists.
2022-12-19 12:23:55+00:00 Use the existing image.
2022-12-19 12:23:57+00:00 Submitting deployment to compute.
2022-12-19 12:24:01+00:00 Checking the status of deployment aciservice..
2022-12-19 12:26:45+00:00 Checking the status of inference endpoint aciservice.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [70]:
# Getting the service state
# The scorig URI & the primary authentication key are copied to the endpoint.py file in order to test the deployed service.
# The Swagger URI can be used in Swagger UI: https://petstore.swagger.io/ For more info, please see the relevant part in the README file.

# # Authentication is enabled, so I use the get_keys method to retrieve the primary and secondary authentication keys:
# primary, secondary = service.get_keys()

print('Service state: ' + service.state)
print('Service scoring URI: ' + service.scoring_uri)
print('Service Swagger URI: ' + service.swagger_uri)
print('Service primary authentication key: ' + primary)


Service state: Healthy
Service scoring URI: http://5e9ae7fd-d333-4fe6-9940-987e1eeec211.southcentralus.azurecontainer.io/score
Service Swagger URI: http://5e9ae7fd-d333-4fe6-9940-987e1eeec211.southcentralus.azurecontainer.io/swagger.json
Service primary authentication key: LK8G9vCuz2XoG2cLqlOjCyFRQRwc3zWx


In the cell below, send a request to the web service you deployed to test it.

In [71]:
#%run endpoint.py

import requests
import json

# URL for the web service, should be similar to:

scoring_uri = ''

# If the service is authenticated, set the key or token

# key = 'LK8G9vCuz2XoG2cLqlOjCyFRQRwc3zWx'

data = {"data":
        [
          "Age": 25,
            "SystolicBP": 130
            "DiastolicBP": 80
            "BS": 15
            "BodyTemp": 98
            "HeartRate": 86          
      ]
    }
# Convert to JSON string
input_data = json.dumps(data)
with open("data.json", "w") as _f:
    _f.write(input_data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
# headers['Authorization'] = f'Bearer {key}'


# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.json())

{"result": [1, 0]}
Expected result: [true, false], where 'true' means '1' and 'false' means '0' as result in the 'DEATH_EVENT' column


TODO: In the cell below, print the logs of the web service and delete the service

In [72]:
# Printing the logs
print(service.get_logs())

2022-12-19T12:26:21,947010100+00:00 - iot-server/run 
2022-12-19T12:26:21,949085900+00:00 - rsyslog/run 
2022-12-19T12:26:21,964420200+00:00 - gunicorn/run 
2022-12-19T12:26:21,970206600+00:00 | gunicorn/run | 
2022-12-19T12:26:21,973776700+00:00 | gunicorn/run | ###############################################
2022-12-19T12:26:21,980089100+00:00 | gunicorn/run | AzureML Container Runtime Information
2022-12-19T12:26:21,988219400+00:00 | gunicorn/run | ###############################################
2022-12-19T12:26:21,988815700+00:00 - nginx/run 
2022-12-19T12:26:22,009753100+00:00 | gunicorn/run | 
2022-12-19T12:26:22,025280600+00:00 | gunicorn/run | 
2022-12-19T12:26:22,045696200+00:00 | gunicorn/run | AzureML image information: openmpi3.1.2-ubuntu18.04, Materializaton Build:20220930.v4
2022-12-19T12:26:22,057764000+00:00 | gunicorn/run | 
2022-12-19T12:26:22,066567400+00:00 | gunicorn/run | 
2022-12-19T12:26:22,075501400+00:00 | gunicorn/run | PATH environment variable: /azureml-env

## Deleting the service
Putting the deletion of the service in a separate cell to avoid accidentally running the cell before finishing the tasks

In [73]:
service.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
